# 要約 
このJupyterノートブックは、Kaggleの「20の質問」ゲームコンペティションに関するデータを収集、分析、更新することを目的としています。具体的には、Meta Kaggleデータセットを利用して、特定のコンペティションに関連するエピソードの情報やゲームの詳細を取得します。

### 主要な処理フローと使用ライブラリ
1. **ライブラリのインポート**:
   - `requests`: HTTPリクエストを送信するために使用。
   - `json`: JSONデータの処理に使用。
   - `pandas`: データの操作や分析を簡単に行うための主要なライブラリ。
   - `pathlib`: ファイルシステムのパス管理。

2. **データの読み込み**:
   - 既存のゲームデータ（CSV形式）をPandas DataFrameで読み込み、ゲームの詳細を理解します。

3. **API経由でのデータ取得**:
   - `download_game`関数を定義し、KaggleのAPIを通じてゲームの詳細をJSON形式で取得、各チームの質問や回答を整理します。

4. **フィルタリングとデータの整理**:
   - Meta Kaggleデータセットから特定のコンペティション（ID 61247）に関連するエピソードのみを抽出し、検証ゲームを除外します。
   - ジェネレータ関数を使用して大規模なCSVファイルを効率的に処理・フィルタリングします。

5. **データの結合と更新**:
   - 新しいゲームデータを既存のものに追加し、カテゴリーごとの新しいゲームを抽出して整理します。
   - 推測者と回答者のSubmissionIdを追加し、各エピソードIDごとの提出物のチェックを行います。

6. **結果の保存**:
   - 更新されたゲームデータとエージェントデータをCSVファイルとして保存し、後の分析に備えます。

このノートブックは、特にデータの抽出、フィルタリング、整理、および更新に焦点を当てており、参加者がコンペティションのデータを効果的に使用できるように設計されています。このプロセスを通じて、参加者は質問ゲームのパフォーマンス解析や新しい戦略の開発に貢献することが期待されています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に基づいて、機械学習・深層学習の初心者がつまずきそうな専門用語の解説です。特に、マイナーな用語や実務経験がないと馴染みが薄いと思われるものに焦点を当てています。

1. **HTTPリクエスト**:
   - ウェブサーバーにデータを要求するプロトコルの一つ。主に、GET（データ取得）やPOST（データ送信）を使ってウェブから情報を取得したり、データを送信したりします。

2. **JSON（JavaScript Object Notation）**:
   - データをテキスト形式で交換するための軽量なフォーマット。人間にも読みやすく、コンピュータでも簡単に解析できるため、APIレスポンスなどで広く使用されています。

3. **データフレーム**:
   - `pandas`ライブラリで使用されるデータ構造。異なる型のデータを列に持つ2次元の表形式で、行や列名を持ち、操作が容易です。

4. **フィルタリング**:
   - データセットから特定の条件を満たすデータを抽出する処理。例えば、特定の列の値に基づいて行を選択することを指します。

5. **インデックス**:
   - データ構造において、データの位置を識別するためのラベルや番号。特に、データフレームでは行または列のラベルとして使用されることが多いです。

6. **ジェネレータ**:
   - メモリ効率が良い方法で一度に大きなデータを処理するためのPythonの機能。データを少しずつ生成し、必要になった時にだけ計算できるため、大きなファイルを扱う際に便利です。

7. **チャンク**:
   - 大きなデータを小さな部分に分けて処理すること。一度にメモリに読み込むデータの量を抑えるために用いられます。

8. **推測者チーム / 回答者チーム**:
   - 「20の質問」ゲームにおける役割。推測者はターゲットワードを当てるための質問を行い、回答者は与えられた質問に「はい」または「いいえ」で答えます。

9. **API**:
   - アプリケーションプログラミングインターフェースの略で、ソフトウェアコンポーネント同士が対話するための規約のこと。ここでは、外部のデータを取得するために使用されています。

10. **マジックコマンド**:
    - Jupyter Notebook特有のコマンドで、特定の操作を実行するために使用されます。例えば、`%%time`コマンドを使うと、セルの実行時間を計測できます。

11. **マージ**:
    - データフレームを統合するプロセス。共通のキーを基に異なるデータフレームを結合することを指します。

12. **SubmissionId**:
    - コンペティションにおいて、参加者が提出したモデルや結果に対して付与される識別子。このIDを使用することで、異なる提出物を追跡・比較できます。

これらの用語は、コンペティションやデータ処理に特有であり、初心者にとって理解が難しい可能性がありますので、丁寧に説明しました。

---


In [ ]:
# requestsライブラリをインポートします。
# HTTPリクエストを送信するための便利なツールです。
import requests

# jsonライブラリをインポートします。
# JSON（JavaScript Object Notation）形式のデータを扱うためのライブラリです。
import json

# pandasライブラリをインポートします。
# データの操作や分析を簡単にするためのライブラリです。
import pandas as pd

# pathlibからPathクラスをインポートします。
# ファイルシステムのパスを扱うための便利な方法を提供します。
from pathlib import Path

[Meta kaggleデータセット](https://www.kaggle.com/datasets/kaggle/meta-kaggle)を使用して、コンペティション「https://www.kaggle.com/competitions/llm-20-questions」からepisodeIdを取得します。  
（meta-kaggleが更新されると、ソースに`[Dataset no longer available]`と表示されることがありますが、それでも機能します）

更新: 推測者と回答者からのsubmissionIdを追加しました。  
次のステップを参照してください: https://www.kaggle.com/code/waechter/llm-20-questions-leaderbord-analyze-best-agents

⚠️ *絶対にフォークして最初から実行しないでください。多くのリクエストが発生しますので、  
代わりにデータセットやノートブックの出力を使用してください* ⚠️

# 前回のゲームをロードする

In [ ]:
# pandasを使用してCSVファイルを読み込みます。
# 指定されたパスからデータをDataFrameとして読み込みます。
# 'index'列をインデックスとして使用します。
df_games = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv", index_col='index')

# 読み込んだデータフレームを表示します。
# この時点で、ゲームに関するデータがどのようなものかを確認できます。
df_games

In [ ]:
# ゲームの情報をダウンロードする関数を定義します。
# 引数にはゲーム番号(num)と作成時間(CreateTime)を使用します。
# do_printがTrueの場合には、ダウンロードしたデータを印刷します。
def download_game(num, CreateTime, do_print=False):
    # APIのURLを設定します。
    url_api = "https://www.kaggleusercontent.com/episodes/{num_episode}.json"
    try:
        # HTTP GETリクエストを送信してデータを取得します。
        r = requests.get(url_api.format(num_episode=num))
        
        # ステータスコードが200（成功）でない場合はエラーメッセージを表示します。
        if r.status_code != 200:
            print(f"Error http {r.status_code=} {num=}")
            return {}
        
        # 取得したレスポンスをJSON形式で解析します。
        resp = r.json()
        
        # チーム名を取得します。最初の2つが推測者チーム、残りが回答者チームとなります。
        teams = resp["info"]["TeamNames"][0:2], resp["info"]["TeamNames"][2:]  # <=> teams=(guesser_team1, answerer_team1), (guesser_team2, answerer_team2)
        formated = {}  # フォーマットされたゲームデータを格納する辞書です。
        
        # 各チームの最後のステップのデータを取得します。
        for (guesser, answerer), step in zip(teams, resp["steps"][-1][1::2]):  # 各チームの最後のステップのみを確認します。
            # キーワードとカテゴリを取得します。
            keyword, category = step['observation']['keyword'], step['observation']['category']
            # do_printがTrueの場合は詳細を表示します。
            if do_print:
                print(f"###\n{guesser=} {answerer=} step {len(step['observation']['answers'])}")
                print(f"{keyword=} {category=}")
                # 各質問、回答、推測を表示します。
                for answer, question, guess in zip(step['observation']['answers'], step['observation']['questions'], step['observation']['guesses']):
                    print(f"\t{question=} {answer=} {guess=}")
            
            # フォーマットされたデータを辞書に追加します。
            game_index = f"{num}__{guesser}__{answerer}"
            formated[game_index] = {key: step['observation'][key] for key in ['answers', 'questions', 'guesses', 'keyword', 'category']}
            formated[game_index]["guesser"] = guesser
            formated[game_index]["answerer"] = answerer
            formated[game_index]["nb_round"] = len(step['observation']['answers'])  # ラウンド数を取得します。
            formated[game_index]["game_num"] = num  # ゲーム番号を追加します。
            formated[game_index]["guessed"] = keyword in step['observation']['guesses']  # 推測したかどうかを確認します。
            formated[game_index]["CreateTime"] = CreateTime  # 作成時間を追加します。
        
        return formated  # フォーマットされたデータを返します。
    except Exception as e:
        print("Error parsing", num)  # エラーが発生した場合、メッセージを表示します。
        # print(e)  # エラーの詳細情報を表示します（必要に応じて）。
        return {}  # 空の辞書を返します。

# meta kaggleデータセットからこのコンペティションの新しいエピソードを取得します。

In [ ]:
# %%timeは、セルの実行にかかる時間を計測するためのマジックコマンドです。
# この後の処理がどれだけ時間がかかったかを表示します。
%%time

# meta_pathには、meta kaggleデータセットのパスを設定します。
meta_path = Path("/kaggle/input/meta-kaggle")

# Episodes.csvファイルを読み込みます。
# インデックスとして'Id'列を使用します。
episodes_df = pd.read_csv(meta_path / "Episodes.csv", index_col="Id")

In [ ]:
# episodes_dfから、CompetitionIdが61247の行のみをフィルタリングします。
# これにより、特定のコンペティションのエピソードデータのみが残ります。
episodes_df = episodes_df.loc[episodes_df.CompetitionId == 61247]

# フィルタリング後のデータフレームを表示します。
# これにより、指定したコンペティションに関連するエピソードの情報を確認できます。
episodes_df

* タイプ4は検証ゲームです（1チーム、両サイド同じエージェント）。
* タイプ1は2チーム、4エージェントのゲームであり、これらのみを保持します。

In [ ]:
# 実際のゲームのみを保持し、検証ゲームをフィルタリングします。
# Typeが1の行のみを選択し、"CreateTime"列だけを残します。
episodes_df = episodes_df.loc[episodes_df.Type == 1, "CreateTime"]

## ジェネレータを使用して大きなファイルを読み込み、フィルタリングします
`EpisodeAgents.csv`ファイルは大きすぎて一度に読み込むことができません。

In [ ]:
# ジェネレータ関数を定義します。
# この関数は大きなCSVファイルを分割して読み込み、特定のエピソードIDに基づいてフィルタリングします。
def filter_gen(filepath=meta_path / "EpisodeAgents.csv", chunksize=10 ** 6):
    # フィルタリング対象のエピソードIDを取得します。
    episodes_ids = episodes_df.index
    # pandasを使用してCSVファイルをチャンク単位で読み込みます。
    with pd.read_csv(filepath, chunksize=chunksize, index_col="Id") as reader:
        for chunk in reader:
            # 現在のチャンク内でEpisodeIdがエピソードIDのリストに含まれているかをマスクします。
            mask = chunk.EpisodeId.isin(episodes_ids)
            # マスクが真となる行があれば、そのチャンクを返します。
            if mask.any():
                yield chunk.loc[mask]  # フィルタリングされたデータをジェネレータとして返します。

In [ ]:
# %%timeは、セルの実行にかかる時間を計測するためのマジックコマンドです。
# この後の処理がどれだけ時間がかかったかを表示します。
%%time

# filter_genジェネレータを使用してフィルタリングされたデータを取得し、
# すべてのチャンクを一つのDataFrameに結合します。
agents_df = pd.concat(filter_gen())

# 結合されたDataFrameを表示します。
# これにより、フィルタリングされたエージェントの情報を確認できます。
agents_df

# 新しいゲームを追加します。

In [ ]:
# %%timeは、セルの実行にかかる時間を計測するためのマジックコマンドです。
# この後の処理がどれだけ時間がかかったかを表示します。
%%time

# 既存のゲームデータの行数を表示します。
print(f"previous {len(df_games)=}")

# 新しいゲームデータをダウンロードし、既存のデータフレームに結合します。
# episodes_dfの各エピソードについて、すでにdf_gamesに含まれていないものだけを追加します。
df_games = pd.concat([df_games] + [pd.DataFrame(download_game(episode_id, date)).T for episode_id, date in episodes_df.items() if episode_id not in df_games.game_num.values])

# 更新後のゲームデータの行数を表示します。
print(f"after update {len(df_games)=}")

# 更新されたデータフレームを表示します。
df_games

In [ ]:
# データフレームから、カテゴリが'place'または'things'であるゲームをフィルタリングします。
# これにより、新しいカテゴリのゲームデータのみが抽出されます。
new_category_games = df_games.loc[(df_games.category == 'place') | (df_games.category == 'things')]

# フィルタリングされた新しいカテゴリのゲームデータを表示します。
new_category_games

In [ ]:
# 新しいカテゴリのゲームから、キーワードとカテゴリの組み合わせを集計し、
# 各ユニークなキーワードを小文字に変換してDataFrameを作成します。
keywords_category = pd.DataFrame({keyword.lower(): {"keyword": keyword, "category": category} for keyword, category in new_category_games[["keyword", "category"]].value_counts().index}).T

# インデックスでソートします。
keywords_category = keywords_category.sort_index()

# 結果を"keywords.csv"という名前でCSVファイルとして保存します。
keywords_category.to_csv("keywords.csv")

## SubmissionIdを追加します。

In [ ]:
# %%timeは、セルの実行にかかる時間を計測するためのマジックコマンドです。
# この後の処理がどれだけ時間がかかったかを表示します。
%%time

# df_gamesをゲーム番号でグループ化します。
for name, group in df_games.groupby(by="game_num"):
    # すでにSubmissionIdが存在し、すべてのSubmissionIdが0より大きい場合はスキップします。
    if "guesser_SubmissionId" in group.columns and "answerer_SubmissionId" in group.columns and (group.guesser_SubmissionId > 0).all() and (group.answerer_SubmissionId > 0).all():
        continue  # すでに処理済み
    
    # agents_dfからエピソードIDがnameと一致するエージェントを取得し、インデックスでソートします。
    agents_sub = agents_df[agents_df.EpisodeId == name].sort_values("Index")
    
    # 賞金の提出物が4つない場合はスキップします。
    if len(agents_sub) != 4:
        print(f"{name=} Not all submissions are available. skip")
        continue
        
    # EpisodeAgentsはエピソードIDごとに4行持ち、インデックスは0-3です。
    # 出力ログのJSONと同じ順序（guesser_team1, answerer_team1, guesser_team2, answerer_team2）です。
    for i, (index, row) in enumerate(group.iterrows()):
        # 各チームごとに2行処理します。
        df_games.loc[index, ["guesser_SubmissionId", "answerer_SubmissionId"]] = [
            agents_sub.loc[agents_sub.Index == i * 2, "SubmissionId"].iloc[0], 
            agents_sub.loc[agents_sub.Index == 1 + i * 2, "SubmissionId"].iloc[0]
        ]

# SubmissionIdの欠損値を0で埋め、整数型に変換します。
df_games[["guesser_SubmissionId", "answerer_SubmissionId"]] = df_games[["guesser_SubmissionId", "answerer_SubmissionId"]].fillna(0).astype("int")

# 更新されたデータフレームを表示します。
df_games

### マージが正しく行われたか確認します。

In [ ]:
# 提出物のIDが、推測者と回答者で一致するか確認します。
if set(df_games.answerer_SubmissionId) != set(df_games.guesser_SubmissionId):
    # 推測者にしかない提出物があれば、それを表示します。
    print("sub only in guesser", set(df_games.guesser_SubmissionId) - set(df_games.answerer_SubmissionId))
    # 回答者にしかない提出物があれば、それを表示します。
    print("sub only answerer", set(df_games.answerer_SubmissionId) - set(df_games.guesser_SubmissionId))

# チームが推測者と回答者で一致するか確認します。
if set(df_games.answerer) != set(df_games.guesser):
    # 推測者にしかないチームがあれば、それを表示します。
    print("team only in guesser", set(df_games.guesser) - set(df_games.answerer))
    # 回答者にしかないチームがあれば、それを表示します。
    print("team only in answerer", set(df_games.answerer) - set(df_games.guesser))

In [ ]:
# ユニークなチーム数と提出物の数を表示します。
# answerer列に含まれるユニークなチームの数と、answerer_SubmissionId列に含まれるユニークな提出物の数を確認します。
f"Nb unique team: {len(df_games.answerer.unique())} submissions: {len(df_games.answerer_SubmissionId.unique())}"

In [ ]:
# コメントアウトされたコードは、回答者のSubmissionIdに対して、推測者と回答者のチーム名が一意でない場合をチェックします。
# 各回答者のSubmissionIdについて、対応するチーム名を取得し、チーム名が複数ある場合は警告を表示します。

# for submissionId in df_games.answerer_SubmissionId.unique():
#     guesser = df_games.loc[df_games.guesser_SubmissionId == submissionId, "guesser"].unique()
#     answerer = df_games.loc[df_games.answerer_SubmissionId == submissionId, "answerer"].unique()
#     if len(answerer) > 1 or len(guesser) > 1:
#         print(f"{submissionId=} found multiple team name {guesser=}, {answerer=}")

submissionId=0は見つからなかったものであり、次のデータ更新時に再度埋め込むことを試みます。  
残りは、チームが名前を更新したものであると思われます。

In [ ]:
# 推測者のSubmissionIdとチーム名の組み合わせの出現回数をカウントします。
# これにより、どのSubmissionIdがどのチーム名に関連付けられているかを確認できます。
df_games[["guesser_SubmissionId", "guesser"]].value_counts()

In [ ]:
# 各回答者チームについて、推測者と回答者のSubmissionIdの共通部分を取得し、その数を表示します。
# これにより、各チームが持つ提出物の数を確認できます。
for team in df_games.answerer.unique():
    submissions_set = set(df_games.loc[df_games.guesser == team, "guesser_SubmissionId"]) & set(df_games.loc[df_games.answerer == team, "answerer_SubmissionId"])
    print(f"{team=}, {len(submissions_set)} submissions")

# 保存します。

In [ ]:
# ゲームデータを"LLM-20Questions-games.csv"という名前でCSVファイルとして保存します。
# インデックスラベルには"index"を使用します。
df_games.to_csv("LLM-20Questions-games.csv", index_label="index")

# エージェントデータを"EpisodeAgents.csv"という名前でCSVファイルとして保存します。
# インデックスラベルには"Id"を使用します。
agents_df.to_csv("EpisodeAgents.csv", index_label="Id")

https://www.kaggle.com/code/waechter/llm-20-questions-leaderbord-analyze-best-agentsで使用します。

---

# コメント

> ## OminousDude
> 
> とても役に立ちました。この素晴らしい作業のおかげで、自分のモデルのパフォーマンスを把握する手助けになりました。ありがとうございます！

> ## Mario Caesar
> 
> LLMデータセットに関する素晴らしい作業ですね。あなたの作業から多くの新しいことを学びました[@waechter](https://www.kaggle.com/waechter)。共有してくれてありがとう😎👊！アップボートしました!! これからも良い仕事を続けてください。

> ## Zeeshan Latif
> 
> [@waechter](https://www.kaggle.com/waechter) 素晴らしい仕事です！

> ## JuHyeon_
> 
> わあ、素晴らしい仕事です！情報豊富な作業を共有してくれてありがとう[@waechter](https://www.kaggle.com/waechter) 
> 
> LLM初心者にとって素晴らしい教材です :)
> 
> アップボートしました😀 時間があれば私のノートブックにも訪れてくれると嬉しいです！

> ## Sara Taghizadeh Milani
> 
> これは多くの思考と努力が注がれたことが明らかです。徹底性と明瞭さに対してアップボートしました—あなたのさらなる作業を楽しみにしています！
> 
> [@waechter](https://www.kaggle.com/waechter) 

> ## Marília Prata
> 
> あなたのデータセット「LLM-20Questions-games」で素晴らしい仕事をしています、Waechter。

---